In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [4]:
import numpy as np
import pandas as pd
import os

# Загрузка данных

In [5]:
input_path = './input_data'
file = os.path.join(input_path,'sensors.csv')
target_file = os.path.join(input_path,'coke_target.csv')

In [6]:
sensors = pd.read_csv(file, index_col="timestamp", parse_dates=["timestamp"])

In [23]:
sensors.head(2)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_38,f_39,f_40,f_41,f_42,f_43,f_44,f_45,f_46,f_47
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-24 16:00:00,99.106443,0.240229,26.415117,-0.005383,6.251910,0.022304,1.466704,11.173872,236232.328494,239141.739468,...,48.786063,46.420238,0.694998,0.143421,0.104411,10518.438958,235581.702442,10271.912950,1.718926,64589.460605
2016-10-24 17:00:00,99.106443,0.240229,26.415117,0.103383,6.465077,0.011693,1.267009,12.987551,236736.358537,237598.686177,...,48.786063,46.420238,0.694998,0.143421,0.104411,10529.380416,236303.564486,10185.329909,18.454216,64736.370856


In [24]:
sensors.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13272 entries, 2016-10-24 16:00:00 to 2018-04-30 15:00:00
Data columns (total 48 columns):
f_0     13272 non-null float64
f_1     13272 non-null float64
f_2     13272 non-null float64
f_3     13272 non-null float64
f_4     13272 non-null float64
f_5     13272 non-null float64
f_6     13272 non-null float64
f_7     13272 non-null float64
f_8     13272 non-null float64
f_9     13272 non-null float64
f_10    13272 non-null float64
f_11    13272 non-null float64
f_12    13272 non-null float64
f_13    13272 non-null float64
f_14    13272 non-null float64
f_15    13272 non-null float64
f_16    13272 non-null float64
f_17    13272 non-null float64
f_18    13272 non-null float64
f_19    13272 non-null float64
f_20    13272 non-null float64
f_21    13272 non-null float64
f_22    13272 non-null float64
f_23    13272 non-null float64
f_24    13272 non-null float64
f_25    13272 non-null float64
f_26    13272 non-null float64
f_27    13272 non-n

# Пропуски в данных

In [25]:
TR_END = "2017-12-31 23:00:00"

In [26]:
sensors.ffill(inplace=True)
sensors.fillna(sensors[:TR_END].mean(), inplace=True)

In [27]:
sensors.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13272 entries, 2016-10-24 16:00:00 to 2018-04-30 15:00:00
Data columns (total 48 columns):
f_0     13272 non-null float64
f_1     13272 non-null float64
f_2     13272 non-null float64
f_3     13272 non-null float64
f_4     13272 non-null float64
f_5     13272 non-null float64
f_6     13272 non-null float64
f_7     13272 non-null float64
f_8     13272 non-null float64
f_9     13272 non-null float64
f_10    13272 non-null float64
f_11    13272 non-null float64
f_12    13272 non-null float64
f_13    13272 non-null float64
f_14    13272 non-null float64
f_15    13272 non-null float64
f_16    13272 non-null float64
f_17    13272 non-null float64
f_18    13272 non-null float64
f_19    13272 non-null float64
f_20    13272 non-null float64
f_21    13272 non-null float64
f_22    13272 non-null float64
f_23    13272 non-null float64
f_24    13272 non-null float64
f_25    13272 non-null float64
f_26    13272 non-null float64
f_27    13272 non-n

# Базовые признаки

In [28]:
features = []

for period in [12, 24, 48, 168]:
    rolling_avg = (sensors
                   .rolling("%iH" % period)
                   .mean()
                   .rename(lambda x: "_".join([x, "%ih" % period, "avg"]), axis=1))
    rolling_std = (sensors
                   .rolling("%iH" % period)
                   .std()
                   .rename(lambda x: "_".join([x, "%ih" % period, "std"]), axis=1))
    features.append(rolling_avg)
    features.append(rolling_std)

In [29]:
features = pd.concat(features, axis=1)
features = sensors.join(features)

In [30]:
features["dayofweek"] = features.index.dayofweek
features["month"] = features.index.month

In [40]:
features

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_40_168h_std,f_41_168h_std,f_42_168h_std,f_43_168h_std,f_44_168h_std,f_45_168h_std,f_46_168h_std,f_47_168h_std,dayofweek,month
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-10-24 16:00:00,99.106443,0.240229,26.415117,-0.005383,6.251910,0.022304,1.466704,11.173872,236232.328494,239141.739468,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10
2016-10-24 17:00:00,99.106443,0.240229,26.415117,0.103383,6.465077,0.011693,1.267009,12.987551,236736.358537,237598.686177,...,0.000000,0.000000,0.000000,7.736779,510.433547,61.223455,11.833637,103.881235,0,10
2016-10-24 18:00:00,99.106443,0.240229,26.415117,-0.004852,6.315273,-0.003072,1.216848,12.824779,235780.400580,238313.969270,...,0.000000,0.000000,0.000000,30.155755,734.917900,62.840000,8.731748,74.827337,0,10
2016-10-24 19:00:00,99.106443,0.240229,26.415117,0.131502,6.437954,-0.012240,1.162371,6.678317,236091.381252,238923.480515,...,0.000000,0.000000,0.000000,27.757317,600.724991,52.253753,7.131191,138.109388,0,10
2016-10-24 20:00:00,98.912665,0.336871,27.897405,0.018193,6.217860,0.015535,1.287526,5.741191,234844.641480,237358.261098,...,0.000000,0.000000,0.000000,24.577134,690.430193,74.738270,6.204980,236.201523,0,10
2016-10-24 21:00:00,98.912665,0.336871,27.897405,0.008601,6.344010,-0.021353,1.207278,6.435199,235558.499102,237018.823072,...,0.000000,0.000000,0.000000,22.662162,672.396189,70.632435,5.562886,237.693198,0,10
2016-10-24 22:00:00,98.912665,0.336871,27.897405,0.018685,6.225590,-0.001246,1.290169,10.870847,235862.845812,238057.711604,...,0.000000,0.000000,0.000000,32.395576,666.777688,65.749383,5.078201,296.938462,0,10
2016-10-24 23:00:00,98.912665,0.336871,27.897405,-0.010295,5.875992,0.006621,1.416750,14.940021,235593.747483,238126.440451,...,0.000000,0.000000,0.000000,30.901579,619.338490,60.951805,4.934714,328.872734,0,10
2016-10-25 00:00:00,98.912665,0.336871,27.897405,0.017258,5.746826,-0.016867,1.262933,10.862439,235060.172170,238174.955327,...,0.000000,0.000000,0.000000,30.086274,580.275973,58.251954,4.711951,520.915950,1,10


# Построение моделей

In [31]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

ImportError: No module named 'catboost'

In [32]:
np.random.seed(382745)

In [33]:
features_tr = features[:"2017-10-31 23:00:00"].copy()
features_cv = features["2017-11-01 00:00:00":"2017-12-31 23:00:00"].copy()
features_ts = features["2018-01-01 00:00:00":].copy()

In [34]:
features_tr = features_tr[~features_tr.isnull().any(axis=1)]

In [35]:
cv = TimeSeriesSplit(3)

In [36]:
params = {"alpha":[1e3, 1e4, 1e5], "l1_ratio":[0.8, 0.9]}

In [37]:
target_pairs = zip(("coke", "reactor_pressure", "he_temp", "he_pressure", "activity"),
                   ("coke_target.csv", "reactor_pressure_target.csv", "temp_on_he_target.csv",
                    "pressure_he_target.csv", "activity_target.csv"))

In [39]:
models = {}

for ti, (target_name, target) in enumerate(target_pairs):
    print("Таргет:", target_name)
    target = pd.read_csv(target_file, index_col="timestamp")
    tr_fts = features_tr.join(target)
    cv_fts = features_cv.join(target)

    gs = GridSearchCV(ElasticNet(),
                      param_grid=params,
                      cv=cv,
                      scoring="neg_mean_squared_error",
                      n_jobs=1,
                      verbose=1)
    gs.fit(tr_fts[features_ts.columns], tr_fts["target"])
    models[target_name] = {"model":gs}
    
    trpr = gs.predict(tr_fts[features_ts.columns])
    cvpr = gs.predict(cv_fts[features_ts.columns])

    print("RMSE на тренировочном множестве:", np.sqrt(mean_squared_error(tr_fts["target"], trpr)))
    print("RMSE на cv множестве:", np.sqrt(mean_squared_error(cv_fts["target"], cvpr)))

    tspr = gs.predict(features_ts)
    submission = pd.DataFrame(tspr, index=features_ts.index, columns=["target"])
    submission.to_csv("baseline_%s.csv" % target_name)    

Таргет: reactor_pressure
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.1s finished


RMSE на тренировочном множестве: 0.5251839739646155
RMSE на cv множестве: 0.2791938841999826
Таргет: he_temp
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.6s finished


RMSE на тренировочном множестве: 0.5251839739646155
RMSE на cv множестве: 0.2791938841999826
Таргет: he_pressure
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.3s finished


RMSE на тренировочном множестве: 0.5251839739646155
RMSE на cv множестве: 0.2791938841999826
Таргет: activity
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.0s finished


RMSE на тренировочном множестве: 0.5251839739646155
RMSE на cv множестве: 0.2791938841999826
